import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io, filters, transform
import os
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

# Training data

In [21]:
#go to folder with 500 images
os.chdir('train500')
X_train = np.array([io.imread(str(i)+'.jpg') for i in range(1,501)])
os.chdir('..')

In [22]:
train_json = json.load(open('train.json'))

In [23]:
y_train = []
for i in range(1,501):
    labels = train_json['annotations'][i]['labelId']
    labels = np.array(list(map(int,labels)))
    y_train.append(labels)
y_train = np.array(y_train)

In [24]:
#pad labels with zeros
y_train_pad = []

for i in y_train:
    num_zeros = 16 - len(i)
    zeros = [0] * num_zeros
    zeros = np.array(zeros)
    new = np.append(i,zeros)
    y_train_pad.append(new)
y_train_pad = np.array(y_train_pad)

In [25]:
y_train_pad[0]

array([ 36,  66,  44, 214, 105, 133,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0])

Resize images to fit (200,128,3) and flatten the resized images

In [26]:
print('smallest vertical:',min(i.shape[0] for i in X_train))
print('smallest horizontal:',min(i.shape[1] for i in X_train))

smallest vertical: 200
smallest horizontal: 128


In [27]:
#All images resized to the smallest dimensions
X_train_resized = [transform.resize(img,(200,128,3)) for img in X_train]
X_train_flat = np.array([img.flatten() for img in X_train_resized])

/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


# Validation data

In [28]:
os.chdir('val500')
X_test = np.array([io.imread(str(i)+'.jpg') for i in range(1,501)])
os.chdir('..')

In [29]:
X_test_resized = [transform.resize(img,(200,128,3)) for img in X_test]
X_test_flat = np.array([img.flatten() for img in X_test_resized])

/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [30]:
val_json = json.load(open('validation.json'))

In [31]:
y_test = []
for i in range(1,501):
    labels = val_json['annotations'][i]['labelId']
    labels = np.array(list(map(int,labels)))
    y_test.append(labels)
y_test = np.array(y_test)

# pad labels with nan
y_test_pad = []
for i in y_test:
    num_zeros = 16 - len(i)
    zeros = [np.nan] * num_zeros
    zeros = np.array(zeros)
    new = np.append(i,zeros)
    y_test_pad.append(new)
y_test_pad = np.array(y_test_pad)

# KNN

In [49]:
knn = KNeighborsClassifier()
knn.fit(X_train_flat,y_train_pad)
pred = knn.predict(X_test_flat)

In [33]:
pred

array([[ 52, 106, 153, ...,   0,   0,   0],
       [ 91,  66,  53, ...,   0,   0,   0],
       [ 11,  17,  66, ...,   0,   0,   0],
       ...,
       [  6,  17,  66, ...,   0,   0,   0],
       [ 17, 222,  66, ...,   0,   0,   0],
       [ 36,  66,  53, ...,   0,   0,   0]])

In [34]:
#pred = pred.astype(np.float)

In [35]:
#replace zeros with nan
#pred[pred == 0] = np.nan

In [48]:
pred

array([[ 52., 106., 153., ...,  nan,  nan,  nan],
       [ 91.,  66.,  53., ...,  nan,  nan,  nan],
       [ 11.,  17.,  66., ...,  nan,  nan,  nan],
       ...,
       [  6.,  17.,  66., ...,  nan,  nan,  nan],
       [ 17., 222.,  66., ...,  nan,  nan,  nan],
       [ 36.,  66.,  53., ...,  nan,  nan,  nan]])

In [37]:
#from sklearn.metrics import accuracy_score
#acc = accuracy_score(y_test_pad, pred)

In [38]:
#Hamming loss https://stackoverflow.com/questions/38697982/python-scikit-learn-multi-class-multi-label-performance-metrics
acc = np.sum(np.not_equal(y_test_pad, pred))/float(y_test_pad.size)
acc


0.9705

# Write Predictions to file

In [51]:
with open("pred500.txt","w") as f:
    # write the header of the file
    f.write("id,predicted\n")
    # assuming the images are sorted according to their IDs, then: number = id+1
    for number, img in enumerate(X_train):
        id_ = number + 1
        f.write(str(id_) + ",")
        for number, label in enumerate(pred[number]):
            if label == 0:
                continue
            # every label but the last is followed by a whitespace
            if number != len(pred[number])-1:
                f.write(str(label) + " ")
            else:
                f.write(str(label))
        f.write("\n")  

In [39]:
f1_score(y_test_pad[0], pred[0], average='micro') 

ValueError: continuous is not supported

In [ ]:
a = np.array([1,2,3,np.nan])

In [ ]:
b = np.array([1,2,3,np.nan,np.nan])

In [ ]:
a == b